In [5]:
import pandas as pd
from ydata_profiling import ProfileReport
from IPython.display import IFrame

# Cargar dataset
df = pd.read_csv("UCI_Credit_Card.csv")

# Crear el reporte
profile = ProfileReport(
    df,
    title="Análisis Exploratorio - UCI Credit Card",
    explorative=True
)

# Guardar reporte en HTML
profile.to_file("Reporte_UCI_Credit_Card.html")

# Mostrar el HTML dentro del Notebook
IFrame(src="Reporte_UCI_Credit_Card.html", width=1000, height=600)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]